In [5]:
import re
import time
import pyperclip

def extract_values(text: str) -> str:
    # 1. Split into lines, drop empty ones
    lines = [ln for ln in text.splitlines() if ln.strip()]

    # 2. For each line, remove everything up to and including the first “:”
    #    then normalize non-numeric entries to “-”
    values = []
    for ln in lines:
        data = re.sub(r'^.*?:\s*', '', ln)
        if not re.search(r'\d', data):
            data = '-'
        values.append(data)

    # 3. Join as a comma-separated string and adjust if it contains “/”
    result = ','.join(values)
    if "/" in result:
        result = "," + result + ",-"
    return result

def watch_clipboard(poll_interval: float = 0.5):
    last = None
    while True:
        current = pyperclip.paste()
        if current != last:
            transformed = extract_values(current)
            pyperclip.copy(transformed)
            print(f"Clipboard updated → {transformed}")
            last = transformed
        time.sleep(poll_interval)

if __name__ == "__main__":
    print("Watching clipboard…  (Ctrl-C to quit)")
    watch_clipboard()


Watching clipboard…  (Ctrl-C to quit)
Clipboard updated → -
Clipboard updated → 0,-10,0,0,0,0,0,0
Clipboard updated → -,-,-,-,-,-
Clipboard updated → 0,-10,0,0,0,0,0,40
Clipboard updated → ,154 / 252 / 542 / 999,154 / 252 / 542 / 999,154 / 252 / 542 / 999,154 / 252 / 542 / 999,-,-,-
Clipboard updated → 10,10,10,35,20,20,20,40
Clipboard updated → ,-,-,-,154 / 252 / 542 / 999,-,-,-
Clipboard updated → 10,10,10,10,40,40,0,20
Clipboard updated → ,154 / 252 / 542 / 999,154 / 252 / 542 / 999,252 / 542 / 999,252 / 542 / 999,154 / 252 / 542 / 999,-,-
Clipboard updated → 10,0,10,10,20,20,40,20
Clipboard updated → ,252 / 542 / 999,252 / 542 / 999,252 / 542 / 999,542 / 999,-,-,-
Clipboard updated → 10,10,10,10,40,40,40,40
Clipboard updated → ,252 / 542 / 999,252 / 542 / 999,252 / 542 / 999,-,-,-,-


KeyboardInterrupt: 

## Tables

In [33]:
#!/usr/bin/env python3
import csv, re, time
from urllib.parse import urlsplit, urlunsplit, quote
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup

ATTACK_KEYS = ["Phy", "Mag", "Fire", "Ligt", "Holy"]
SCALE_KEYS  = ["STR", "DEX", "INT", "FAI", "ARC"]

# ----------------- HTTP helpers -----------------
def normalize_url(url: str) -> str:
    p = urlsplit(url)
    return urlunsplit((p.scheme, p.netloc, quote(p.path), quote(p.query, safe="=&"), p.fragment))

def session_with_retries():
    s = requests.Session()
    r = Retry(total=6, backoff_factor=0.9,
              status_forcelist=[429,500,502,503,504],
              allowed_methods=["GET","HEAD","OPTIONS"])
    s.headers.update({"User-Agent":"Mozilla/5.0 (compatible; NightreignScraper/1.5)"})
    s.mount("https://", HTTPAdapter(max_retries=r))
    s.mount("http://",  HTTPAdapter(max_retries=r))
    return s

S = session_with_retries()

def fetch(url: str, timeout=25) -> str:
    u = normalize_url(url)
    r = S.get(u, timeout=timeout)
    r.raise_for_status()
    return r.text

# ----------------- Parse helpers -----------------
def _txt_nl(el): return el.get_text("\n", strip=True) if el else ""
def _norm_label(s: str) -> str:
    s = re.sub(r"[\s\xa0]+"," ", s or "").strip().lower()
    s = re.sub(r"[:·•]+$","", s)
    return s

def find_infobox_table(soup: BeautifulSoup):
    box = soup.select_one("#infobox, div.infobox")
    if not box:
        return None
    return box.select_one("table")

def is_attack_td(td) -> bool:
    # Icon title is very reliable on Fextralife
    if td.find("img", attrs={"title": re.compile(r"attack power", re.I)}):
        return True
    txt = _txt_nl(td)
    # Accept only if the cell STARTS with "Attack" or "Attack Power"
    return re.match(r"^\s*attack(\s+power)?\b", txt, re.I) is not None

def parse_attack_td(td):
    block = td.find(class_="lineleft") or td
    txt = _txt_nl(block)
    # normalize Lightning variants to your "Ligt" column
    txt = re.sub(r"\b(Lightning|Light)\b", "Ligt", txt, flags=re.I)
    # drop leading "Attack"/"Attack Power"
    txt = re.sub(r"^\s*Attack(\s+Power)?\s*", "", txt, flags=re.I)
    pairs = dict(re.findall(r"(Phy|Mag|Fire|Ligt|Holy)\s*([+-]?\d+)", txt, flags=re.I))
    out = {k: pairs.get(k) or pairs.get(k.lower()) or "-" for k in ATTACK_KEYS}
    return out

def parse_scaling_block(text: str):
    norm = {
        "STRENGTH":"STR","STR":"STR",
        "DEXTERITY":"DEX","DEX":"DEX",
        "INTELLIGENCE":"INT","INT":"INT",
        "FAITH":"FAI","FAI":"FAI",
        "ARCANE":"ARC","ARC":"ARC",
    }
    vals = {}
    lines = [x.strip().upper() for x in text.split("\n") if x.strip()]
    if lines and lines[0].startswith("SCALING"):
        lines = lines[1:]
    i = 0
    while i < len(lines):
        m = re.match(r"^(STR|DEX|INT|FAI|ARC|STRENGTH|DEXTERITY|INTELLIGENCE|FAITH|ARCANE)\s+([SABCDEF])$", lines[i])
        if m:
            vals[norm[m.group(1)]] = m.group(2); i += 1; continue
        if lines[i] in norm and i+1 < len(lines) and re.match(r"^[SABCDEF]$", lines[i+1]):
            vals[norm[lines[i]]] = lines[i+1]; i += 2; continue
        i += 1
    for k in SCALE_KEYS:
        vals.setdefault(k, "-")
    return vals

def extract_fields(html: str):
    soup = BeautifulSoup(html, "html.parser")
    table = find_infobox_table(soup)
    if not table:
        return None

    out = {
        "Weapon Class":"-","Weapon Name":"-",
        "Attack Affinity":"-","Status Ailment":"-","Unique Weapon Effect":"-",
        "Attack Phy":"-","Attack Mag":"-","Attack Fire":"-","Attack Ligt":"-","Attack Holy":"-",
        "Scaling STR":"-","Scaling DEX":"-","Scaling INT":"-","Scaling FAI":"-","Scaling ARC":"-",
    }

    # Name
    h2 = table.find("h2")
    out["Weapon Name"] = (h2.get_text(strip=True) if h2 else (table.find("th").get_text(strip=True) if table.find("th") else "-"))

    attack_done = False

    for tr in table.find_all("tr", recursive=True):
        cells = tr.find_all(["th","td"], recursive=False)
        if len(cells) < 2:
            continue

        left_raw, right_raw = _txt_nl(cells[0]), _txt_nl(cells[1])
        left_norm = _norm_label(left_raw)

        # --- Attack Power row (avoid 'attack affinity') ---
        if not attack_done and (is_attack_td(cells[0]) or is_attack_td(cells[1])):
            attack_td = cells[0] if is_attack_td(cells[0]) else cells[1]
            att = parse_attack_td(attack_td)
            for k in ATTACK_KEYS:
                out[f"Attack {k}"] = att[k]
            attack_done = True
            continue

        # --- Scaling + class row ---
        if left_norm.startswith("scaling"):
            sc = parse_scaling_block(left_raw)
            for k in SCALE_KEYS:
                out[f"Scaling {k}"] = sc[k]
            if right_raw:
                out["Weapon Class"] = right_raw
            continue

        # --- Simple key:value rows ---
        if left_norm == "attack affinity":
            out["Attack Affinity"] = right_raw or "-"
            continue
        if left_norm == "status ailment":
            out["Status Ailment"] = right_raw or "-"
            continue
        if left_norm == "unique weapon effect":
            out["Unique Weapon Effect"] = right_raw or "-"
            continue

    return out

# ----------------- CSV driver -----------------
def to_csv(urls, csv_path="weapons_extract.csv", delay=1.0):
    header = ["URL","Weapon Class","Weapon Name","Attack Affinity","Status Ailment","Unique Weapon Effect",
              "Attack Phy","Attack Mag","Attack Fire","Attack Ligt","Attack Holy",
              "Scaling STR","Scaling DEX","Scaling INT","Scaling FAI","Scaling ARC"]
    rows = [header]

    for url in urls:
        try:
            html = fetch(url)
            fields = extract_fields(html) or {}
            row = [url] + [fields.get(col, "-") for col in header[1:]]
            rows.append(row)
            print(f"[OK] {url}")
        except Exception as e:
            rows.append([url] + ["-"]*(len(header)-1))
            print(f"[ERR] {url} -> {e}")
        time.sleep(delay)

    with open(csv_path, "w", encoding="utf-8", newline="") as f:
        csv.writer(f).writerows(rows)
    print(f"Saved {csv_path}")


# Load all URLs from weapon_urls.txt (one per line, skip blanks/#)
with open("weapon_urls.txt", "r", encoding="utf-8") as fh:
    urls = [ln.strip() for ln in fh if ln.strip() and not ln.strip().startswith("#")]

to_csv(urls, "weapons_extract.csv", delay=0.1)


[OK] https://eldenringnightreign.wiki.fextralife.com/Black+Knife
[OK] https://eldenringnightreign.wiki.fextralife.com/Blade+of+Calling
[OK] https://eldenringnightreign.wiki.fextralife.com/Bloodstained+Dagger
[OK] https://eldenringnightreign.wiki.fextralife.com/Celebrant's+Sickle
[OK] https://eldenringnightreign.wiki.fextralife.com/Cinquedea
[OK] https://eldenringnightreign.wiki.fextralife.com/Crystal+Knife
[OK] https://eldenringnightreign.wiki.fextralife.com/Dagger
[OK] https://eldenringnightreign.wiki.fextralife.com/Duchess'+Dagger
[OK] https://eldenringnightreign.wiki.fextralife.com/Erdsteel+Dagger
[OK] https://eldenringnightreign.wiki.fextralife.com/Glintstone+Kris
[OK] https://eldenringnightreign.wiki.fextralife.com/Great+Knife
[OK] https://eldenringnightreign.wiki.fextralife.com/Ivory+Sickle
[OK] https://eldenringnightreign.wiki.fextralife.com/Misericorde
[OK] https://eldenringnightreign.wiki.fextralife.com/Reduvia
[OK] https://eldenringnightreign.wiki.fextralife.com/Parrying+Dagg

### Download Images Individually

In [51]:
#!/usr/bin/env python3
import os, re, csv, time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlsplit, urlunsplit, quote, unquote, urljoin

# ================= HTTP helpers =================

def absolutize_img_src(src: str, page_url: str) -> str:
    s = (src or "").strip()
    if not s:
        return ""
    # Fix cases like "https:///eldenringnightreign..."
    if s.startswith("https:///"):
        s = s.replace("https:///", "https://", 1)
    if s.startswith("http:///"):
        s = s.replace("http:///", "http://", 1)

    # already absolute
    if re.match(r"^https?://", s, re.I):
        return s
    if s.startswith("//"):
        return "https:" + s.lstrip("/")
    return urljoin(page_url, s)
def normalize_url(url: str) -> str:
    p = urlsplit(url)
    # if someone handed us "https:///host/path" (empty netloc, host in path), fix it
    if p.scheme and not p.netloc and p.path.startswith("//"):
        fixed = "https://" + p.path.lstrip("/")
        p = urlsplit(fixed)
    return urlunsplit((p.scheme, p.netloc, quote(p.path), quote(p.query, safe="=&"), p.fragment))

def session_with_retries():
    from requests.adapters import HTTPAdapter
    from urllib3.util.retry import Retry
    s = requests.Session()
    r = Retry(
        total=6,
        backoff_factor=0.9,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD", "OPTIONS"]
    )
    s.headers.update({"User-Agent": "Mozilla/5.0 (compatible; NightreignScraper/1.6)"})
    s.mount("https://", HTTPAdapter(max_retries=r))
    s.mount("http://",  HTTPAdapter(max_retries=r))
    return s

S = session_with_retries()

def fetch(url: str, timeout=25) -> str:
    u = normalize_url(url)
    r = S.get(u, timeout=timeout)
    r.raise_for_status()
    return r.text

# ================= Filename helpers =================
INVALID_FS = r'<>:"/\\|?*\0'  # basic Windows-unsafe chars

def weapon_name_from_url(url: str) -> str:
    """Take the last path component, decode, clean, Title Case, fix possessives ('S -> 's)."""
    path = urlsplit(url).path.strip("/")
    last = unquote(path.split("/")[-1] if path else "")
    last = last.replace("+", " ").replace("_", " ").replace("’", "'")
    last = re.sub(r"\s+", " ", last).strip()
    titled = last.title()
    titled = re.sub(r"'S\b", "'s", titled)   # Knight'S -> Knight's
    return titled or "Unknown Weapon"

def safe_filename(basename: str, ext=".png", keep_apostrophe=True) -> str:
    """Make OS-safe filename; by default preserves apostrophes and possessive 's."""
    s = unquote(basename or "").strip().replace("’", "'")
    s = re.sub(r"\s+", " ", s)
    if keep_apostrophe:
        # ensure Title Case + possessives are correct
        s = re.sub(r"'S\b", "'s", s.title())
    else:
        # drop apostrophes cleanly -> Knights
        s = re.sub(r"'s\b", "s", s, flags=re.I).replace("'", "").title()
    s = "".join(ch for ch in s if ch not in INVALID_FS).strip().rstrip(".")
    if not s:
        s = "Unknown Weapon"
    if ext and not s.lower().endswith(ext.lower()):
        s += ext
    return s

# ================= Image discovery =================


def pick_from_srcset(srcset: str) -> str | None:
    """
    srcset like: '...200px.webp 200w, ...400px.webp 400w'
    We pick the last (usually largest).
    """
    parts = [p.strip() for p in (srcset or "").split(",") if p.strip()]
    if not parts:
        return None
    last = parts[-1]
    cand = last.split()[0]
    return cand

def find_best_image_url(html: str, page_url: str) -> str | None:
    soup = BeautifulSoup(html, "html.parser")

    # 1) OpenGraph
    og = soup.find("meta", attrs={"property": "og:image"})
    if og and og.get("content"):
        return absolutize_img_src(og["content"], page_url)

    # 2) <link rel="image_src">
    link_img = soup.find("link", rel=lambda v: v and "image_src" in v)
    if link_img and link_img.get("href"):
        return absolutize_img_src(link_img["href"], page_url)

    # helper to filter out junky icons
    def acceptable(img):
        src = (img.get("src") or img.get("data-src") or "")
        alt = (img.get("alt") or "")
        junk = r"(icon|attack|defense|status|scaling|spinner|blank|pixel)"
        return not re.search(junk, (src + " " + alt), re.I)

    # 3) Prefer image inside the infobox
    box = soup.select_one("#infobox, div.infobox")
    if box:
        imgs = [img for img in box.find_all("img") if acceptable(img)]
        if imgs:
            img = imgs[0]
            src = pick_from_srcset(img.get("srcset")) or img.get("src") or img.get("data-src")
            if src:
                return absolutize_img_src(src, page_url)
        # fallback to any img in the box
        any_img = box.find("img")
        if any_img:
            src = pick_from_srcset(any_img.get("srcset")) or any_img.get("src") or any_img.get("data-src")
            if src:
                return absolutize_img_src(src, page_url)

    # 4) First acceptable image on the page
    for img in soup.find_all("img"):
        if not acceptable(img):
            continue
        src = pick_from_srcset(img.get("srcset")) or img.get("src") or img.get("data-src")
        if src:
            return absolutize_img_src(src, page_url)

    return None

def guess_ext_from_headers_or_url(resp, img_url: str) -> str:
    ctype = resp.headers.get("Content-Type", "").lower()
    if "image/webp" in ctype: return ".webp"
    if "image/png" in ctype: return ".png"
    if "image/jpeg" in ctype or "image/jpg" in ctype: return ".jpg"
    # fallback from URL
    for ext in (".webp", ".png", ".jpg", ".jpeg", ".gif"):
        if img_url.lower().split("?")[0].endswith(ext):
            return ".jpg" if ext == ".jpeg" else ext
    return ".png"

# ================= Downloader =================
IMG_OUT_DIR = "weapon_images"
os.makedirs(IMG_OUT_DIR, exist_ok=True)

def download_weapon_image(url: str, keep_apostrophe=True) -> tuple[bool, str]:
    """
    Returns (success, message). On success, message is output path (or 'exists -> path').
    On failure, message is an error string.
    """
    try:
        html = fetch(url)
        img_url = find_best_image_url(html, url)
        if not img_url:
            return False, "no-image-found"

        # Request the image
        resp = S.get(normalize_url(img_url), timeout=25, stream=True)
        resp.raise_for_status()

        wname = weapon_name_from_url(url)
        ext = guess_ext_from_headers_or_url(resp, img_url)
        out_name = safe_filename(wname, ext=ext, keep_apostrophe=keep_apostrophe)
        out_path = os.path.join(IMG_OUT_DIR, out_name)

        # Skip if already present and non-empty
        if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
            return True, f"exists -> {out_path}"

        with open(out_path, "wb") as f:
            for chunk in resp.iter_content(8192):
                if chunk:
                    f.write(chunk)
        return True, out_path
    except Exception as e:
        return False, str(e)

def download_all_images(url_list: list[str], delay=0.2, keep_apostrophe=True):
    ok, fail = 0, 0
    for u in url_list:
        u = u.strip()
        if not u or u.startswith("#"):
            continue
        success, msg = download_weapon_image(u, keep_apostrophe=keep_apostrophe)
        if success:
            ok += 1
            print(f"[IMG OK]  {u} -> {msg}")
        else:
            fail += 1
            print(f"[IMG ERR] {u} -> {msg}")
        time.sleep(delay)
    print(f"\nImage download complete. OK: {ok} | ERR: {fail}")

# ================= Main =================
if __name__ == "__main__":
    # Reads one URL per line from weapon_urls.txt
    with open("weapon_urls.txt", "r", encoding="utf-8") as fh:
        img_urls = [ln for ln in fh if ln.strip() and not ln.strip().startswith("#")]
    download_all_images(img_urls, delay=0.01, keep_apostrophe=True)


[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Black+Knife -> weapon_images\Black Knife.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Blade+of+Calling -> weapon_images\Blade Of Calling.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Bloodstained+Dagger -> weapon_images\Bloodstained Dagger.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Celebrant's+Sickle -> weapon_images\Celebrant's Sickle.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Cinquedea -> weapon_images\Cinquedea.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Crystal+Knife -> weapon_images\Crystal Knife.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Dagger -> weapon_images\Dagger.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Duchess'+Dagger -> weapon_images\Duchess' Dagger.png
[IMG OK]  https://eldenringnightreign.wiki.fextralife.com/Erdsteel+Dagger -> weapon_images\Erdsteel Dagger.png
[IMG OK]  https://eldenring

### Extract Links

In [32]:
import re, time, csv
from urllib.parse import urlsplit, urlunsplit, urljoin, urldefrag, quote
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup

SEED = "https://eldenringnightreign.wiki.fextralife.com/Weapons"
OUT  = "weapon_urls.csv"
DELAY = 0.1

def normalize_url(url: str) -> str:
    p = urlsplit(url)
    path  = quote(p.path)
    query = quote(p.query, safe="=&")
    return urlunsplit((p.scheme, p.netloc, path, query, p.fragment))

def session_with_retries():
    s = requests.Session()
    r = Retry(total=6, backoff_factor=0.7,
              status_forcelist=[429,500,502,503,504],
              allowed_methods=["GET","HEAD","OPTIONS"])
    s.headers.update({"User-Agent": "Mozilla/5.0 (compatible; NRLinkGrab/1.0)"})
    s.mount("https://", HTTPAdapter(max_retries=r))
    s.mount("http://",  HTTPAdapter(max_retries=r))
    return s

S = session_with_retries()

def fetch(url, timeout=25):
    u = normalize_url(url)
    r = S.get(u, timeout=timeout)
    r.raise_for_status()
    return r.text

def find_infobox_table(soup: BeautifulSoup):
    box = soup.select_one("#infobox, div.infobox")
    return (box.select_one("table") if box else None)

def looks_like_weapon_page(html: str) -> bool:
    soup = BeautifulSoup(html, "html.parser")
    table = find_infobox_table(soup)
    if not table:
        return False
    txt = table.get_text(" ", strip=True).lower()
    return any(m in txt for m in ("attack affinity", "weapon skill", "unique weapon effect", "level req"))

def clean_href(base, href):
    if not href: return None
    href, _ = urldefrag(href)
    absu = urljoin(base, href)
    p = urlsplit(absu)
    if p.scheme not in ("http","https"): return None
    # ignore file/user/special/map/media pages
    low = absu.lower()
    bad = ("/file/", "/user", "/special:", "/map", "/help", "/Category:".lower())
    if any(b in low for b in bad): return None
    return urlunsplit((p.scheme, p.netloc, p.path, p.query, ""))

# 1) Fetch the Weapons hub page and collect same-site links
html = fetch(SEED)
soup = BeautifulSoup(html, "html.parser")
host = urlsplit(SEED).netloc

scope = soup.select_one("#wiki-content") or soup  # be focused if possible
candidates = []
seen = set()
for a in scope.find_all("a", href=True):
    u = clean_href(SEED, a["href"])
    if not u: continue
    if urlsplit(u).netloc != host: continue
    if u in seen: continue
    seen.add(u)
    candidates.append(u)

print(f"Found {len(candidates)} candidate links. Validating…")

# 2) Validate each link is a weapon page via infobox check
weapon_urls = []
for i,u in enumerate(candidates, 1):
    try:
        page = fetch(u)
        if looks_like_weapon_page(page):
            weapon_urls.append(u)
            print(f"[{i}/{len(candidates)}] ✓ {u}")
        else:
            print(f"[{i}/{len(candidates)}] – {u}")
    except Exception as e:
        print(f"[{i}/{len(candidates)}] ERR {u} -> {e}")
    time.sleep(DELAY)

weapon_urls = list(dict.fromkeys(weapon_urls))
print(f"\nDiscovered {len(weapon_urls)} weapon pages.")

# 3) Save
with open(OUT, "w", encoding="utf-8", newline="") as f:
    w = csv.writer(f)
    for u in weapon_urls:
        w.writerow([u])

print(f"Saved to {OUT}")


Found 413 candidate links. Validating…
[1/413] – https://eldenringnightreign.wiki.fextralife.com/Weapons
[2/413] – https://eldenringnightreign.wiki.fextralife.com/Elden+Ring+Nightreign+Wiki
[3/413] – https://eldenringnightreign.wiki.fextralife.com/Classes
[4/413] – https://eldenringnightreign.wiki.fextralife.com/Armor
[5/413] – https://eldenringnightreign.wiki.fextralife.com/Skills
[6/413] – https://eldenringnightreign.wiki.fextralife.com/Multiplayer+Coop+and+Online
[7/413] – https://eldenringnightreign.wiki.fextralife.com/Game+Progress+Route
[8/413] ERR https://eldenringnightreign.wiki.fextralife.com/todo -> 404 Client Error: Not Found for url: https://eldenringnightreign.wiki.fextralife.com/todo
[9/413] – https://eldenringnightreign.wiki.fextralife.com/General+Information
[10/413] – https://eldenringnightreign.wiki.fextralife.com/Patch+Notes
[11/413] – https://eldenringnightreign.wiki.fextralife.com/Network+Test
[12/413] – https://eldenringnightreign.wiki.fextralife.com/DLC
[13/413] 